In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [2]:
# 加载Iris数据集
iris = load_iris()
data = iris.data
target = iris.target
# 将target转换成one-hot编码
target = torch.eye(3)[target]

In [3]:
# 划分训练集和测试集
data_train, data_test, target_train, target_test = train_test_split(
    data, target, test_size=0.3)

In [4]:
# 定义神经网络
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 5)
        self.fc2 = nn.Linear(5, 5)
        self.fc3 = nn.Linear(5, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.softmax(x, dim=1)

In [5]:
# 定义训练函数
def train(net, X, Y, optimizer, criterion):
    net.train()  # 将模型设为训练模式
    optimizer.zero_grad()  # 梯度清零，避免累加
    output = net(X)  # 前向传播
    loss = criterion(output, Y)  # 计算损失
    loss.backward()  # 反向传播
    optimizer.step()  # 更新模型参数
    return loss.item()

In [6]:
# 定义测试函数
def test(net, X, Y):
    net.eval()  # 将模型设为测试模式
    with torch.no_grad():  # 关闭梯度计算
        output = net(X)
        pred = torch.argmax(output, dim=1)  # 取预测概率最大的类别
        acc = torch.mean((pred == torch.argmax(Y, dim=1)).float())  # 计算准确率
    return acc.item()

In [7]:
net = Net()
learning_rate = 0.1
criterion = nn.BCELoss()  # 二分类交叉熵损失
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

In [8]:
epochs = 5000
train_losses = []
test_losses = []
test_accs = []

在代码中先导入了`Sklearn`库中的`load_iris`和`train_test_split`函数，用于载入Iris数据集并将数据集划分为训练集和测试集。然后在定义神经网络时，使用了`nn.Linear`定义了3个全连接层，其中输入层有4个神经元，第一和第二个隐藏层分别有5个神经元，输出层有3个神经元。在训练神经网络时，使用了BCELoss（二分类交叉熵损失）作为损失函数，SGD（随机梯度下降优化器）来优化。每一轮训练后，会计算训练集和测试集的损失和准确率，并输出监控训练进程。

In [9]:
for i in range(epochs):
    train_loss = train(net, torch.tensor(data_train, dtype=torch.float32),
                       torch.tensor(target_train, dtype=torch.float32),
                       optimizer, criterion)
    train_losses.append(train_loss)

    test_loss = criterion(net(torch.tensor(data_test, dtype=torch.float32)),
                          torch.tensor(target_test, dtype=torch.float32))
    test_losses.append(test_loss.item())

    test_acc = test(net, torch.tensor(data_test, dtype=torch.float32),
                    torch.tensor(target_test, dtype=torch.float32))
    test_accs.append(test_acc)

    if (i+1) % 100 == 0:
        print('Epoch [{}/{}], train_loss: {:.4f}, test_loss: {:.4f}, test_acc: {:.4f}'
              .format(i+1, epochs, train_loss, test_loss.item(), test_acc))

/tmp/ipykernel_112307/1158182061.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(target_train, dtype=torch.float32),
/tmp/ipykernel_112307/1158182061.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(target_test, dtype=torch.float32))
/tmp/ipykernel_112307/1158182061.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(target_test, dtype=torch.float32))


Epoch [100/5000], train_loss: 0.3696, test_loss: 0.3488, test_acc: 0.8667
Epoch [200/5000], train_loss: 0.2536, test_loss: 0.2405, test_acc: 0.9778
Epoch [300/5000], train_loss: 0.1476, test_loss: 0.1203, test_acc: 1.0000
Epoch [400/5000], train_loss: 0.1592, test_loss: 0.0993, test_acc: 0.9111
Epoch [500/5000], train_loss: 0.1120, test_loss: 0.0582, test_acc: 1.0000
Epoch [600/5000], train_loss: 0.0609, test_loss: 0.0402, test_acc: 1.0000
Epoch [700/5000], train_loss: 0.0526, test_loss: 0.0400, test_acc: 1.0000
Epoch [800/5000], train_loss: 0.0504, test_loss: 0.0400, test_acc: 0.9778
Epoch [900/5000], train_loss: 0.0430, test_loss: 0.0365, test_acc: 0.9778
Epoch [1000/5000], train_loss: 0.0430, test_loss: 0.0378, test_acc: 0.9778
Epoch [1100/5000], train_loss: 0.0391, test_loss: 0.0367, test_acc: 0.9778
Epoch [1200/5000], train_loss: 0.3182, test_loss: 0.2735, test_acc: 0.8444
Epoch [1300/5000], train_loss: 0.0371, test_loss: 0.0372, test_acc: 0.9778
Epoch [1400/5000], train_loss: 0.0